In [1]:
import torch

torch.manual_seed(0)
import numpy as np

In [2]:
B, J, N = 1, 2, 5  # batch size, betas, agents

# Simulate typical network outputs

w = torch.rand(B, J, N)
A_diag = torch.rand(B, J, N) + 0.1  # avoid zeros to stay positive definite
b = torch.randn(B, J, N)
c = torch.randn(B, J)

In [3]:
w * A_diag

tensor([[[0.2228, 0.3854, 0.0108, 0.0355, 0.1211],
         [0.3922, 0.3909, 0.8068, 0.1189, 0.2417]]])

In [4]:
(w * (A_diag * w))

tensor([[[0.1105, 0.2961, 0.0010, 0.0047, 0.0372],
         [0.2487, 0.1916, 0.7233, 0.0542, 0.1528]]])

In [5]:
# sum of Ai * wi^2
(A_diag * w**2).sum(dim=2)

tensor([[0.4495, 1.3706]])

In [6]:
print(A_diag.shape)

torch.Size([1, 2, 5])


In [7]:
quad_terms_manual = torch.zeros((B, J))

quad_sum = 0
for i in range(B):
    for j in range(J):
        quad_sum = 0.0
        for k in range(N):
            quad_sum += A_diag[i, j, k].item() * (w[i, j, k].item() ** 2)
        quad_terms_manual[i, j] = quad_sum

quad_terms_manual

tensor([[0.4495, 1.3706]])

In [8]:
(A_diag * w.pow(2)).sum(dim=2)

tensor([[0.4495, 1.3706]])

In [9]:
import os, sys


def get_dir_n_levels_up(path, n):
    # Go up n levels from the given path
    for _ in range(n):
        path = os.path.dirname(path)
    return path


proj_root = get_dir_n_levels_up(os.path.abspath("__file__"), 3)
sys.path.append(proj_root)

from opinion_dynamics.models import OpinionNet

B, J, N = 2, 2, 3
model = OpinionNet(nr_agents=N, nr_betas=J)
model.train()

print(model.predict_A_b_c.weight.grad) 

x = torch.randn(B, N)
out = model(x)

loss = out["c"].sum()
loss.backward()

model.predict_A_b_c.weight.grad 

None


tensor([[0.1487, 0.0910, 0.0000, 0.0000, 0.0000, 0.0726, 0.4755, 0.0319, 0.7576,
         0.7684, 0.0216, 0.2214, 0.0000, 0.0000, 0.2769, 0.0950, 0.0000, 0.3722,
         0.0000, 0.0000, 0.2417, 0.1467, 0.2461, 0.0000, 0.4756, 0.0049, 0.0000,
         0.0231, 0.0974, 0.0929, 0.5306, 0.0206, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.3482, 0.1504, 0.0000, 0.2402, 0.3626, 0.7391, 0.5719,
         0.1945, 0.0000, 0.2136, 0.7041, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.2467, 0.3349, 0.1573, 0.0000, 0.2079, 0.5339, 0.1077, 0.0000, 0.4445,
         0.5485],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0

In [10]:
model.predict_A_b_c.weight.shape

torch.Size([14, 64])